In [1]:
from Datasets import TokenizedChromaDataset, PermutationsTokenizedChromaDataset
import numpy as np
from torch.utils.data import DataLoader, Subset
import sys
sys.path.insert(0, '..')
from transformer.models import TransformerFromModels, EncoderModel, DecoderModel
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
import torch
from tqdm import tqdm
import os

In [2]:
npz_path = '../data/augmented_and_padded_data.npz'
dataset = TokenizedChromaDataset(npz_path)

train_percentage = 0.9
split_idx = int( len(dataset)*train_percentage )

train_set = Subset(dataset, range(0,split_idx))
test_set = Subset(dataset, range(split_idx, len(dataset)))

batch_size = 16
epochs = 1000

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

In [3]:
permutation_dataset = PermutationsTokenizedChromaDataset(npz_path)
permutation_loader = DataLoader(permutation_dataset, batch_size=batch_size, shuffle=True)

In [4]:
src_vocab_size = 2**12
tgt_vocab_size = 2**12
d_model = 256
num_heads = 4
num_layers = 4
d_ff = 256
max_seq_length = 129
dropout = 0.3

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

encoderModel = EncoderModel(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
decoderModel = DecoderModel(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

encoderModel = encoderModel.to(dev)
decoderModel = decoderModel.to(dev)

transformer = TransformerFromModels(encoderModel, decoderModel)

transformer = transformer.to(dev)

In [5]:
criterion = CrossEntropyLoss(ignore_index=0)
optimizer = Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

# keep best validation loss for saving
best_val_loss = np.inf
save_dir = '../saved_models/encoder_decoder_one_hot/'
encoder_path = save_dir + 'encoder_one_hot.pt'
decoder_path = save_dir + 'decoder_one_hot.pt'
os.makedirs(save_dir, exist_ok=True)

for epoch in range(epochs):
    train_loss = 0
    running_loss = 0
    samples_num = 0
    running_accuracy = 0
    accuracy = 0
    with tqdm(train_loader, unit='batch') as tepoch:
        tepoch.set_description(f"Epoch {epoch} | trn")
        for melodies, chords in tepoch:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            optimizer.zero_grad()
            output = transformer(melodies, chords[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords[:, 1:].contiguous().view(-1))
            loss.backward()
            optimizer.step()
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            train_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords[:, 1:]).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
            tepoch.set_postfix(loss=train_loss, accuracy=accuracy) # tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
    with tqdm(permutation_loader, unit='batch') as tepoch:
        tepoch.set_description(f"Epoch {epoch} | prm")
        for melodies, chords in tepoch:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            optimizer.zero_grad()
            output = transformer(melodies, chords[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords[:, 1:].contiguous().view(-1))
            loss.backward()
            optimizer.step()
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            train_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords[:, 1:]).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
            tepoch.set_postfix(loss=train_loss, accuracy=accuracy) # tepoch.set_postfix(loss=loss.item(), accuracy=100. * accuracy)
    # validation
    with torch.no_grad():
        val_loss = 0
        running_loss = 0
        samples_num = 0
        running_accuracy = 0
        accuracy = 0
        print('validation...')
        for melodies, chords in test_loader:
            melodies = melodies.to(dev)
            chords = chords.to(dev)
            output = transformer(melodies, chords[:, :-1])
            loss = criterion(output.contiguous().view(-1, tgt_vocab_size), chords[:, 1:].contiguous().view(-1))
            # update loss
            samples_num += melodies.shape[0]
            running_loss += loss.item()
            val_loss = running_loss/samples_num
            # accuracy
            prediction = output.argmax(dim=2, keepdim=True).squeeze()
            running_accuracy += (prediction == chords[:, 1:]).sum().item()/prediction.shape[1]
            accuracy = running_accuracy/samples_num
        if best_val_loss > val_loss:
            print('saving!')
            best_val_loss = val_loss
            torch.save(encoderModel.state_dict(), encoder_path)
            torch.save(decoderModel.state_dict(), decoder_path)
        print(f'validation: accuracy={accuracy}, loss={val_loss}')
    # print(f"Epoch: {epoch+1}, training loss: {train_loss} | validation loss {val_loss}")

Epoch 0 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:49<00:00,  6.01batch/s, accuracy=0.0156, loss=0.375]
Epoch 0 | prm: 100%|█████████████████████████████████████████████████████████████████████████████████| 333/333 [00:55<00:00,  5.97batch/s, accuracy=0.0238, loss=0.369]


validation...
saving!
validation: accuracy=0.04464704502814259, loss=0.32719523106014975


Epoch 1 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:49<00:00,  6.01batch/s, accuracy=0.0572, loss=0.281]
Epoch 1 | prm: 100%|█████████████████████████████████████████████████████████████████████████████████| 333/333 [00:55<00:00,  5.97batch/s, accuracy=0.0492, loss=0.307]


validation...
saving!
validation: accuracy=0.0572672373358349, loss=0.29777783792864315


Epoch 2 | trn: 100%|█████████████████████████████████████████████████████████████████████████████████| 300/300 [00:49<00:00,  6.01batch/s, accuracy=0.0698, loss=0.253]
Epoch 2 | prm: 100%|█████████████████████████████████████████████████████████████████████████████████| 333/333 [00:55<00:00,  5.97batch/s, accuracy=0.0586, loss=0.287]


...

validation...
saving!
validation: accuracy=0.09020286116322701, loss=0.23549226703608014


Epoch 30 | trn: 100%|███████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:49<00:00,  6.00batch/s, accuracy=0.127, loss=0.147]
Epoch 30 | prm: 100%|███████████████████████████████████████████████████████████████████████████████████████| 333/333 [00:55<00:00,  5.97batch/s, accuracy=0.105, loss=0.199]


validation...
saving!
validation: accuracy=0.09039340994371482, loss=0.23286743996067297


Epoch 31 | trn:  98%|████████████████████████████████████████████████████████████████████████████████████▉  | 293/300 [00:49<00:01,  5.97batch/s, accuracy=0.128, loss=0.145]


KeyboardInterrupt: 

In [ ]:
# to load the model
encoderModel = EncoderModel(src_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)
decoderModel = DecoderModel(tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

encoderModel.load_state_dict(torch.load(encoder_dir))
decoderModel.load_state_dict(torch.load(decoder_dir))

transformer = TransformerFromModels(encoderModel, decoderModel)
# for inference, not retraining, we need to run
transformer.eval()